In [1]:
%load_ext autoreload
%autoreload 2
%pylab
%matplotlib inline

import os
import sys
import pandas as pd
import pickle
import numpy as np

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
from batch_sv_system_utils import compute_eer
from batch_sv_system_utils import compute_error

In [165]:
np.random.rand(3,5)[-1:]

array([[0.79424325, 0.19043324, 0.20102014, 0.72356485, 0.81917568]])

In [168]:
def prep_scores(data, size):
    c = 0
    for i, x in enumerate(data):
        if x.shape[0] < size:
            c += 1
            res = size - x.shape[0]
            x = np.concatenate([x, np.repeat(x[-1:], res, 0)])
        data[i] = np.sort(x[:size], axis=0)

In [169]:
train_x, train_y = eval_plda_score_list[:5000], eval_plda_labels[:5000]
test_x, test_y = eval_plda_score_list[5000:], eval_plda_labels[5000:]

prep_scores(train_x, 20)
prep_scores(test_x, 20)

train_x = np.concatenate(train_x, 1).T
test_x = np.concatenate(test_x, 1).T
train_y = np.concatenate(train_y, 0)
test_y = np.concatenate(test_y, 0)

In [91]:
def get_features(x, n_feat=5):
    min_ = x.min(0)
    max_ = x.max(0)
    median_ = np.median(x, axis=0)
    avg_ = x.mean(0)
    std_ = x.std(0)
   
    if n_feat==2:
        return np.stack([avg_, std_], axis=0).T
    elif n_feat==3:
        return np.stack([max_, avg_, std_], axis=0).T
    elif n_feat==4:
        return np.stack([min_, max_, avg_, std_], axis=0).T
    elif n_feat==5:
        return np.stack([min_, max_, median_, avg_, std_], axis=0).T

In [92]:
eval_plda_trial_output = pickle.load(
    open("trials/dev940_eval311/hard_enr3xsess_ntar9/proxy_eT5_fNorm.pkl", "rb"))
eval_plda_score_list = [x[0] for x in eval_plda_trial_output]
eval_plda_adapt_labels = [x[3] for x in eval_plda_trial_output]
eval_plda_labels = [x[2] for x in eval_plda_trial_output]
eval_plda_proxy_score_list = [x[1] for x in eval_plda_trial_output]
eval_plda_proxy_idx = [x[4][:n_adapt] for x in eval_plda_trial_output]

In [212]:
eval_plda_trial_output = pickle.load(
    open("trials/dev940_eval311/random_enr3xsess_ntar9/proxy_eT5.pkl", "rb"))
eval_plda_score_list = [x[0] for x in eval_plda_trial_output]
eval_plda_labels = [x[2] for x in eval_plda_trial_output]

In [213]:
eval_plda_feats = [get_features(x, 4) for x in eval_plda_score_list]

In [226]:
test_x, test_y = eval_plda_score_list, eval_plda_labels

In [185]:
train_x, train_y = eval_plda_feats[:5000], eval_plda_labels[:5000]
test_x, test_y = eval_plda_feats[5000:], eval_plda_labels[5000:]

train_x = np.concatenate(train_x, 0)
test_x = np.concatenate(test_x, 0)
train_y = np.concatenate(train_y, 0)
test_y = np.concatenate(test_y, 0)

In [215]:
import torch

# train_x = torch.from_numpy(train_x).float()
# train_y = torch.from_numpy(train_y).float()
test_x = torch.from_numpy(test_x).float()
test_y = torch.from_numpy(test_y).float()

In [207]:
from torch.utils.data.dataset import Dataset

class ScoreDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __getitem__(self, index):
        return self.x[index], self.y[index]

    def __len__(self):
        return len(self.y)

In [216]:
from torch.utils.data.dataloader import DataLoader

train_set = ScoreDataset(train_x, train_y)
train_dataloader = DataLoader(train_set, batch_size=512, shuffle=True)
test_set = ScoreDataset(test_x, test_y)
test_dataloader = DataLoader(test_set, batch_size=1024, shuffle=False)

In [194]:
import torch.nn as nn

model = nn.Sequential(
    nn.Linear(4, 64),
    nn.ReLU(),
    nn.Linear(64, 1)
)

In [195]:
from torch.optim import SGD
import torch.nn.functional as F

model = model.cuda()
pos_weight_ = (1-train_y.mean())/train_y.mean()
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight_.cuda())
optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-5, nesterov=True)

model.train()
for eporch_i in range(10):
    loss_sum = 0
    total = 0
    for b_idx, (x, y) in enumerate(train_dataloader):
        x = x.cuda()
        y = y.cuda()
        optimizer.zero_grad()
        logit = model(x)
        loss = criterion(logit.squeeze(), y)
        loss.backward()
        loss_sum += loss.item()
        total += len(y)
        optimizer.step()
        if b_idx % 5000 == 1:
            print(loss_sum / total)

0.00256443431135267
0.0005374724511458611
0.0005023196709140337
0.00048698804728573077
0.0004970548034179956
0.0004432937361527655
0.0004416808811451873
0.0004400846161665346
0.0004091158916708082
0.0004310969483095086
0.00042914065005194364
0.00042882006220599345
0.0006994093710090965
0.0004253945279670399
0.00042417484236732106
0.00042395997534008854
0.0005710579280275851
0.000419642891272645
0.0004200523550941033
0.00042103232962374134
0.0003552038542693481
0.00041931306005818904
0.00041868793682679994
0.0004193486394669664
0.00039658238529227674
0.00041799595550785155
0.0004178625329307089
0.0004185890567460893
0.0005420547386165708
0.0004165800311383879
0.0004171398888723662
0.00041761283725664155
0.0003124524373561144
0.0004163402785716604
0.0004164229441461136
0.0004170103435184378
0.00035661413858179003
0.00041900254146370316
0.00041701416306000615
0.0004168971160006393


In [217]:
model.eval()
test_logits = []
test_labels = []
for b_idx, (x, y) in enumerate(test_dataloader):
    x = x.cuda()
    optimizer.zero_grad()
    logit = model(x)
    test_logits.append(logit.cpu().detach().numpy())
    test_labels.append(y.numpy())

In [218]:
test_scores = np.concatenate(test_logits).reshape(-1)
test_labels = np.concatenate(test_labels)

In [219]:
compute_eer(test_scores, test_labels)

(0.5653894639840891, 0.5653878862326372, 0.5653910417355409, 139.39949)

In [227]:
# test_x, test_y = eval_plda_score_list[5000:], eval_plda_labels[5000:]
test_scores = np.concatenate([x.mean(0) for x in test_x])
test_y = np.concatenate(test_y, 0)
compute_eer(test_scores, test_y)

(0.009668225364954465,
 0.009668744621026583,
 0.009667706108882346,
 -3.0254157051025654)